https://keras.io/examples/vision/video_classification/

In [1]:
# === Imports === #

import tensorflow as tf
from tensorflow.keras.models import load_model

from tqdm import tqdm

# Various python packages are used in this notebook. Please get yourself used to them (optional).
import pandas as pd  # used for storing a tabular representation of the dataset, similar to XLS files.
from pathlib import Path # used to check if the saved model files and accessories.
import requests #used to request remote judge.csv evaluation 
from sklearn.preprocessing import StandardScaler  # used for normalization of dataset
from sklearn.preprocessing   import LabelBinarizer    # used for splitting the gender column
from sklearn.preprocessing   import MinMaxScaler      # used for normalization of dataset
from sklearn.model_selection import train_test_split  # used for performing the train-test split of a dataframe
import cv2                                            # OpenCV used for image processing
import random   #random number generator
import datetime #used to get current date/time
import math     #math/numerical functions
import os       #os specific functions, like file open/close etc.
import gc       #garbage collection module -- used to manually clean up memory spaces/references.


from sklearn.preprocessing import OneHotEncoder   #My favorite categorical to numerical feature conversion tool
from tensorflow import keras  # keras used for construction of the Artificial neural network
from keras.models import Model, Sequential #keras model architectures
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D #types of layers
from keras.losses import mean_squared_error, huber, log_cosh  #built-in loss 
from tensorflow.python.keras.saving import hdf5_format  #used for saving models 
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard  #callbacks
from keras.models import model_from_json  #used for loading model architecture from json file
import h5py  #saved model type

import matplotlib.pyplot as plt  # used for training visualization
import numpy as np  # numpy arrays used for matrix computations

# === Extra Configurations for the GPU Environment === #
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0: #If you have at least one "configured" GPU, let's use it; otherwise, pass
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# https://github.com/zaid478/Transfer-Learning-from-Xception-Model-in-Keras-/blob/master/transfer_learn.py

from keras.applications import xception
from keras import backend as K
from keras.utils import np_utils

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

import pickle 

In [2]:
"""
Setting work environment with dataset. If on Google colaboratory, we need to extract dataset stored in google drive,
otherwise the dataset is already there.
"""
try:
    from google.colab import drive
    print('Running on Google colab...')
    drive.mount('/content/drive')
except:
    print('Running on local machine...')

Running on Google colab...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# IMPORTANT PARAMETERS

In [17]:
preprocessor_folder = 'gen-rnn-7'

train_path = 'drive/MyDrive/Colab Notebooks/DAiSEE/'+preprocessor_folder+'/Train/'
test_path = 'drive/MyDrive/Colab Notebooks/DAiSEE/'+preprocessor_folder+'/Test/'

image_shape = (224, 299, 3) # HEIGHT, WIDTH, CHANNELS

#IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 10
NUM_FEATURES = 2048

# Build a feature extractor

In [4]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=image_shape,
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input(image_shape)
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [5]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, path, list_IDs, labels, batch_size=32, dim=(480,640), n_channels=3,
                 n_classes=4, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.path = path
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 4), dtype='float32')

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            filename = os.path.join(self.path, self.list_IDs[ID])
            filehandler = open(filename, 'rb') 
            x = pickle.load(filehandler)
            X[i,] = x
            y[i,:] = tf.one_hot(self.labels[ID][1], depth=4)

        #### DO INFERENCE ON CNN MODEL #####

        # `frame_masks` and `frame_features` are what we will feed to our sequence model.
        # `frame_masks` will contain a bunch of booleans denoting if a timestep is
        # masked with padding or not.
        frame_masks = np.zeros(shape=(self.batch_size, MAX_SEQ_LENGTH), dtype="bool")
        frame_features = np.zeros(
            shape=(self.batch_size, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # For each video.
        for i in range(X.shape[0]):
        #for idx, path in enumerate(video_paths):
            # Gather all its frames and add a batch dimension.
            frames = X[i]
            frames = frames[None, ...]

            # Initialize placeholders to store the masks and features of the current video.
            temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
            temp_frame_features = np.zeros(
                shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
            )

            # Extract features from the frames of the current video.
            for i, batch in enumerate(frames):
                video_length = batch.shape[0]
                length = min(MAX_SEQ_LENGTH, video_length)
                for j in range(length):
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )
                temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

            frame_features[i,] = temp_frame_features.squeeze()
            frame_masks[i,] = temp_frame_mask.squeeze()

        ### END INFERENCE ###

        #print(tf.convert_to_tensor(y).shape)

        return (frame_features, frame_masks), tf.convert_to_tensor(y)

In [6]:
import os
import shutil
#os.getcwd()

temp_data_location = 'temp_data'

if temp_data_location in os.listdir(os.getcwd()):
    shutil.rmtree(temp_data_location)

os.mkdir(temp_data_location)

#assert temp_data_location not in os.listdir(os.getcwd())


local_train_path = os.path.join(temp_data_location, 'train')
local_test_path = os.path.join(temp_data_location, 'test') 

# Copytree makes the folders
#os.mkdir(local_train_path)
#os.mkdir(local_test_path)

shutil.copytree(train_path, local_train_path) 
shutil.copytree(test_path, local_test_path)

'temp_data/test'

In [18]:
# Generator Parameters
params = {'dim': (MAX_SEQ_LENGTH, image_shape[0], image_shape[1]),
          'batch_size': BATCH_SIZE,
          'n_classes': 4,
          'n_channels': 3,
          'shuffle': True}

folder_path = './drive/MyDrive/Colab Notebooks/DAiSEE/'
all_labels = pd.read_csv(os.path.join(folder_path, 'Labels/AllLabels.csv'))
all_labels['ID_num'] = all_labels['ClipID'].str[:-4]

train_labels = pd.read_csv(os.path.join(folder_path, 'Labels/TrainLabels.csv'))
train_labels['ID_num'] = train_labels['ClipID'].str[:-4]

test_labels = pd.read_csv(os.path.join(folder_path, 'Labels/TestLabels.csv'))
test_labels['ID_num'] = test_labels['ClipID'].str[:-4]

# Train Set
file_labels = []
print("building train set")
for filename in tqdm(os.listdir(local_train_path)):
    try:
        #sample_ID = filename[:filename.index('-')]
        sample_ID = filename[:-4]
        label = all_labels[all_labels['ID_num']==sample_ID].values.tolist()[0][1:-1]
        file_labels.append((filename, np.array(label)))
    except IndexError:
        print(sample_ID)

label_arr = np.array(file_labels, dtype=object)
X_train = label_arr[:, 0]
y_train = label_arr[:, 1]

# Test Set
file_labels = []
print("building test set")
for filename in tqdm(os.listdir(local_test_path)):
    try:
        #sample_ID = filename[:filename.index('-')]
        sample_ID = filename[:-4]
        
        label = test_labels[test_labels['ID_num']==sample_ID].values.tolist()[0][1:-1]
        file_labels.append((filename, np.array(label)))
    except IndexError:
        print(sample_ID)

label_arr = np.array(file_labels, dtype=object)
X_test = label_arr[:, 0]
y_test = label_arr[:, 1]

building train set


100%|██████████| 5482/5482 [00:05<00:00, 984.37it/s]


building test set


 18%|█▊        | 331/1866 [00:00<00:00, 1657.68it/s]

9988260143
9988260248
9988260241
9988260137
998826022
9988260264
9988260247
9988260215
9988260254
9988260259
9988260144
9988260273
9988260152
9988260126
9988260165
9988260141
9988260134
998826019
9988260250


...

9988260240
9988260127
9988260212
9988260237
9988260130
9988260138
9988260132
9988260232
9988260276
9988260281
9988260275
9988260150
9988260257
9988260145
9988260224
9988260227
998826017


 73%|███████▎  | 1363/1866 [00:00<00:00, 1663.87it/s]

9988260236
9988260210
9988260214
9988260133
9988260123
9988260217
9988260251
9988260211
998826016
9988260242
9988260245
998826026
9988260228
998826014
9988260279
9988260255
9988260163
9988260160
9988260222
9988260167
9988260246
9988260146
9988260230
9988260139
9988260270
9988260249
9988260277
9988260216
9988260243


100%|██████████| 1866/1866 [00:01<00:00, 1663.88it/s]

998826021
9988260121
9988260239
9988260157
9988260231
9988260269
998826024
9988260268
9988260235
998826013
9988260135
9988260229
9988260154
9988260233
9988260129
9988260234
998826023


In [8]:
X_test.shape

(1784,)

In [9]:
y_test.shape

(1784,)

In [10]:
y_test

array([array([1, 2, 1, 1]), array([0, 3, 1, 0]), array([0, 2, 0, 0]), ...,
       array([1, 3, 0, 0]), array([1, 2, 0, 0]), array([0, 2, 3, 2])],
      dtype=object)

In [19]:
# Generators
training_generator = DataGenerator(local_train_path, X_train, y_train, **params)
validation_generator = DataGenerator(local_test_path, X_test, y_test, **params) 

In [12]:
'''
class_weight = {0: 75.,
                1: 10.,
                2: 1.,
                3: 1.}
'''
class_weight = {0: 1.,
                1: 1.,
                2: 1.,
                3: 1.}

# RNN

In [13]:
%reload_ext tensorboard
model_path = os.path.join(folder_path, 'saved_models/model_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '_.sav')
log_dir = os.path.join(folder_path, "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_cbk = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_cbk = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='min')
mcp_save_cbk = ModelCheckpoint(model_path+'.mcp.hdf5', save_best_only=True, monitor='val_accuracy', mode='min')
reduce_lr_plateau_cbk = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, mode='min')
#callbacks = [early_stopping_cbk, mcp_save_cbk, reduce_lr_plateau_cbk, tensorboard_cbk]
callbacks = [mcp_save_cbk, tensorboard_cbk]

In [14]:
class_vocab = 4

frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# Refer to the following tutorial to understand the significance of using `mask`:
# https://keras.io/api/layers/recurrent_layers/gru/
x = keras.layers.GRU(128, return_sequences=True)(
    frame_features_input, mask=mask_input
)
x = keras.layers.GRU(64)(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(64, activation="relu")(x) # Increase last FC layer nodes??
output = keras.layers.Dense(4, activation="softmax")(x)

rnn_model = keras.Model([frame_features_input, mask_input], output)

#loss = "sparse_categorical_crossentropy" ### Requires integer labels
loss = 'categorical_crossentropy'

rnn_model.compile(
    loss=loss, optimizer="adam", metrics=["accuracy"]
)

In [20]:
history = rnn_model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    batch_size = BATCH_SIZE,
    callbacks=callbacks
)

Epoch 1/50
85/85 [==============================] - 4149s 49s/step - loss: 0.8675 - accuracy: 0.4710 - val_loss: 0.8737 - val_accuracy: 0.4525
Epoch 2/50
85/85 [==============================] - 4072s 48s/step - loss: 0.8652 - accuracy: 0.4811 - val_loss: 0.8723 - val_accuracy: 0.4971
Epoch 3/50
85/85 [==============================] - 4016s 47s/step - loss: 0.8646 - accuracy: 0.4831 - val_loss: 0.8731 - val_accuracy: 0.4919
Epoch 4/50
85/85 [==============================] - 3987s 47s/step - loss: 0.8662 - accuracy: 0.4790 - val_loss: 0.8667 - val_accuracy: 0.4959
Epoch 5/50
85/85 [==============================] - 4013s 47s/step - loss: 0.8650 - accuracy: 0.4818 - val_loss: 0.8652 - val_accuracy: 0.4936
Epoch 6/50
85/85 [==============================] - 4023s 47s/step - loss: 0.8657 - accuracy: 0.4838 - val_loss: 0.8705 - val_accuracy: 0.4965
Epoch 7/50
85/85 [==============================] - 4020s 47s/step - loss: 0.8655 - accuracy: 0.4800 - val_loss: 0.8650 - val_accuracy: 0.4959

FailedPreconditionError: ignored